In [2]:
import numpy as np

In [35]:
#problem 1
class simplexMethod(object):
    def __init__(self, c, A, b):
        if (np.dot(A, zerovec) > b).any():
            raise ValueError ("Problem not feasible at origin!")
        else:
            self.c = c
            self.A = A
            self.b = b
        
        

In [36]:
simp = simplexMethod
simp.A = np.array([[1, -1], [3, 1], [4, 3]]).reshape(3, 2)
simp.b = np.array([[2, 5, -7]]).reshape(3, 1)
simp.c = np.array([[3, 2]]).reshape(2, 1)


In [13]:
simplexMethod.A

array([[ 1, -1],
       [ 3,  1],
       [ 4,  3]])